In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [2]:
#크롤된 뉴스 데이터 불러오기
columns = ['media','date','title','article_original','url']
news_crawled_df = pd.read_csv('./crawled_news/news_df_210205_v02.csv')
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
news_crawled_df.isna().sum()

media                0
date                 0
title                0
article_original    13
url                  0
dtype: int64

In [4]:
news_crawled_df.loc[news_crawled_df.article_original.isna()]

,media,date,title,article_original,url
69,중앙일보,2020-02-12,권혁재의 사람사진 방탄소년단이 연결한 세계,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
142,중앙일보,2020-03-03,기록 경신한 BTS 빌보드 핫 100 4위K팝 그룹 최고 순위,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
167,중앙일보,2020-03-22,민기자야 BTS 기사 쓴 기자가 직접 자기 기사 댓글 읽었다,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
203,조선일보,2020-05-04,BTS도 먹은 달고나커피 멈출 수가 없어요,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
221,조선일보,2020-05-15,사진 BTS RM이 남긴 미술관 방명록쉽지 않은 시기 같이 이겨내요,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
348,조선일보,2020-08-22,BTS 첫 영어곡 발표 빌보드 1위 오를까,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
415,한겨레,2020-09-02,영상 BTS 빌보드 핫 100 1위온라인 글로벌 기자간담회,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
510,한겨레,2020-09-20,문 대통령 방탄소년단 노래 가사 들려 따라갈 수 있겠더라,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
675,한겨레,2020-10-14,BTS 소속사의 주식 상장이 논란에 싸인 이유 간단정리 공모주 내손안의Q 19,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
738,중앙일보,2020-11-02,Hot Poll 방탄소년단 병역특례에 대한 중앙일보 온라인 핫 폴 결과,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
#drop na
news_crawled_df.dropna(inplace=True)
news_crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 0 to 870
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             858 non-null    object
 1   date              858 non-null    object
 2   title             858 non-null    object
 3   article_original  858 non-null    object
 4   url               858 non-null    object
dtypes: object(5)
memory usage: 40.2+ KB


In [6]:
# reset index
news_crawled_df.reset_index(inplace=True)
news_crawled_df.drop('index',inplace=True,axis=1)

In [ ]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None)
stopwords = list(stopwords2[0])
stopwords

In [8]:
#형태소 토크나이징
from konlpy.tag import Okt
okt = Okt()

articles = news_crawled_df.article_original

articles_morphed = []
for article in articles:
#     morph = okt.morphs(article)
    morph = okt.nouns(article)
    
    stopped = ''
    for word in morph:
        if word not in stopwords:
            stopped += word
            stopped += ' '
            
    articles_morphed.append(stopped)

In [9]:
#형태소 분해된 컬럼 추가
news_crawled_df['article_morphed'] = articles_morphed
#날짜 칼럼 추가
news_crawled_df['date_pd']=pd.to_datetime(news_crawled_df['date'])

In [11]:
#원본 date column drop
news_crawled_df.drop(columns='date',axis=1,inplace=True)

In [12]:
#기사 날짜 14일마다 그룹화
date_grouped = []
date_range = pd.date_range(start='2020-01-01', end='2020-12-31',periods=13)
L = len(date_range)
R = len(news_crawled_df)

for i, date in enumerate(date_range):
    index_group = []
    if i< L-1:
        for j in range(R):
            if date_range[i] < news_crawled_df.date_pd[j] <= date_range[i+1]:
                index_group.append(j)
        date_grouped.append(index_group)

In [13]:
date_range

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-31 10:00:00',
               '2020-03-01 20:00:00', '2020-04-01 06:00:00',
               '2020-05-01 16:00:00', '2020-06-01 02:00:00',
               '2020-07-01 12:00:00', '2020-07-31 22:00:00',
               '2020-08-31 08:00:00', '2020-09-30 18:00:00',
               '2020-10-31 04:00:00', '2020-11-30 14:00:00',
               '2020-12-31 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [14]:
# 그룹별 tfidf 변환
def tfidfy(time_period_no):
    tfidf = TfidfVectorizer(lowercase=False)
    #그룹에 있는 아티클 가져오기
    article_group = []
    for index in date_grouped[time_period_no]:
      article_group.append(news_crawled_df.article_original[index])
    #article_group = [news_crawled_df.article_original[i] for i in date_grouped[time_period_no]]
    
    art_group_tfidf = tfidf.fit_transform(article_group)
    return art_group_tfidf

In [15]:
#date 그룹 내에서 유사도 높은 기사의 index 가져오는 함수
def get_similar_art_index(group_no, art_group_tfidf, group_index):
    # 그룹 안의 한 기사와 나머지 기사간의 similarity 비교
    sim_pair = cosine_similarity(art_group_tfidf[group_index],art_group_tfidf)
    # 인덱스 내림차순으로 소트
    sorted_index = sim_pair.argsort()[:,::-1]
    # 자기자신은 빼기
    sorted_index = sorted_index[:,1:]
    #print(sorted_index)
    
    #유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
    art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]
    art_sim_value = art_sim_value[1:]
    #print(art_sim_value)
    
    #기사의 real 인덱스의 reference point 구하기
    ref = 0
    for i in range(group_no):
        ref += len(date_grouped[i])

    # 유사도가 0.3보다 높은 real 인덱스만 뽑아오기.
    index_high = []
    for idx, value in enumerate(art_sim_value):
         if value>=0.2:
            group_index = sorted_index[0][idx]
            real_index = ref + group_index
            index_high.append(real_index)
            #유사 article로 참조된 횟수 카운터
            sim_counter[real_index] += 1
            
    return index_high

In [16]:
#참조 카운터
sim_counter = np.zeros(len(news_crawled_df))
#전체 문서에 대해, 날짜 구간 당 유사한 기사의 index를 리스트에 저장한다.
sim_list=[]
for group_no in range(len(date_grouped)):
  for idx in range(len(date_grouped[group_no])):
    art_group_tfidf = tfidfy(group_no)
    sim_list.append(get_similar_art_index(group_no, art_group_tfidf, idx))

#dataframe에 컬럼 추가
news_crawled_df['sim_articles'] = sim_list
news_crawled_df['sim_count'] = sim_counter

In [ ]:
#유사한 기사가 3개 이상이면 불러오기  --> 없어도 되는 단계
idx_list = []
for i,ls in enumerate(news_crawled_df['sim_articles']):
  if len(ls)>=3:
    idx_list.append(i)
#print(idx_list)
news_crawled_df.iloc[idx_list]

In [18]:
#대표 기사 뽑기 (가장 참조가 많이 된 기사)
articles_final = []
for group in date_grouped:
    indx = group[0]
    for art_indx in group:
        if news_crawled_df.sim_count[art_indx] >= news_crawled_df.sim_count[indx]:
            indx = art_indx
    articles_final.append(indx)
articles_final

[44, 108, 139, 187, 224, 269, 311, 344, 473, 626, 772, 803]

In [19]:
# 뽑은 대표 기사 갯수
len(articles_final)

12

In [20]:
# from gensim.summarization import summarize
from lexrankr import LexRank
# 출력
# 1. 유사 카운트
# 2. 날짜
# 3. 기사 제목
# 4. 기사 본문
# 5. 요약본
for indx in articles_final:
    sim_count = news_crawled_df.sim_count[indx]
    if sim_count>2:
        lexrank = LexRank()
        lexrank.summarize(news_crawled_df.article_original[indx])
        summaries = lexrank.probe(3)
        print('sim_count: ', sim_count)
        print('article index: ', indx)
        print(news_crawled_df.date_pd[indx])
        print(news_crawled_df.title[indx],'\n')
        print(news_crawled_df.article_original[indx],'\n')
        print(summaries,'\n')

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


sim_count:  6.0
article index:  44
2020-01-28 00:00:00
39년 만에 그래미 바꾼 19세 소녀 빌리 BTS는 한국 가수로 첫 공연 

 2001년에 태어난 열아홉 살 소녀 빌리 아일리시가 그래미상의 역사를 다시 썼다. 방탄소년단은 한국 가수 최초로 그래미 무대에서 공연했다. 시상식에 앞선 레드카펫에선 반전反戰 트럼프 지지 등 정치 패션이 물결쳤다. 어느 해보다 이슈가 만발한 그래미 시상식이었다.  26일현지 시각 미국 로스앤젤레스 스테이플스 센터에서 열린 제62회 그래미 어워즈 시상식. 주인공은 2001년생 싱어송라이터 빌리 아일리시였다. 신인상부터 레코드상·앨범상·노래상 등 주요 부문 4개를 싹쓸이했다. 그래미 최초로 21세기에 태어난 본상 수상자다. 1981년 크리스토퍼 크로스의 4개 부문 수상 이후 39년 만. 역대 최연소 올해의 앨범상 수상자였던 테일러 스위프트당시 20세 기록도 깼다.  미국 LA에서 태어났다. 아버지는 배우 패트릭 오코넬 어머니는 배우 겸 가수 매기 베어드다. 네 살 위인 오빠 피니즈 오코넬과 함께 홈스쿨링으로 교육받았다. 네 살 때 처음 곡을 썼다. 2016년 싱글 오션 아이즈가 처음 빌보드에 오르면서 이름을 알렸다. 지난해 데뷔 앨범이 세계 차트를 휩쓸었고 미국 내 전체 판매 2위를 차지했다. 앨범 작업은 오빠와 함께한다. 대표곡 배드 가이는 중독성 있는 후렴구와 기괴한 뮤직비디오가 인상적. 그는 우울증과 외모에 심각한 결점이 있다고 생각하는 신체이형장애와 싸우고 있어 헐렁한 옷차림을 고집한다. 2018년 처음 내한했고 오는 8월 두 번째 내한 공연이 예정돼 있다.  내 가방엔 호미가 들었지I got the homis i my bag호미는 철로 만들고 한국 거고 최고지Homis made of steel from Korea theyre the beeest.  미국 인기 래퍼 릴 나스 엑스21가 자신의 히트곡 올드 타운 로드Old Tow Road를 선창한 뒤 무대가 바뀌자 그룹 방탄소년단BTS

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


sim_count:  14.0
article index:  139
2020-03-02 00:00:00
BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹 

그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔  7MAP OF THE SOUL  7이 영국 오피셜 차트에 이어 빌보드까지 양대 팝 차트 1위에 올랐다. 한국 가수로서는 최초로 오피셜 차트에서 두 번 연속 정상에 오른 것은 물론 빌보드에서는 비틀스 이후 최단기간 동안 4개 앨범을 1위에 올려놓은 가수라는 기록을 쓰게 됐다.  미국 빌보드는 1일현지시각 방탄소년단의 새 앨범 맵 오브 더 솔  7이 메인 앨범 차트인 ‘빌보드 200 최신 차트3월7일자에서 1위에 올랐다고 밝혔다. 이번 앨범은 지난달 28일 영국·독일·프랑스 등의 오피셜 앨범차트 1위에 올랐고 일본 오리콘차트 1위도 차지했다. 유럽과 일본을 넘어 미국까지 글로벌 앨범 시장을 모두 석권한 셈이다.  ‘빌보드 200’은 빌보드 양대 차트‘빌보드 200’ ‘핫 100’ 중 하나로 메인 앨범 순위를 매기는 차트다. 실물 음반 판매량에다 스트리밍 횟수와 디지털 음원 다운로드 횟수를 각각 앨범 판매량으로 환산한 수치를 합산해 순위를 매긴다. 음원 10곡을 다운받거나 1500곡을 스트리밍한 경우 음반 1장을 산 것으로 간주한다.  빌보드가 인용한 닐슨뮤직 데이터의 자료를 보면 지난달 21일 발매된 이 앨범은 27일까지 총 42만2000장의 판매고를 올렸다. 이 가운데 실물 앨범 판매량은 34만7000장이다. 이외에 스트리밍 횟수를 앨범 판매량으로 환산한 수치가 4만8000장 디지털 음원 다운로드 횟수를 앨범 판매량으로 환산한 수치는 2만6000장으로 집계됐다.  방탄소년단은 이번 1위로 아시아에선 처음으로 네 장의 앨범을 연이어 빌보드 정상에 올린 가수가 됐다. 방탄소년단은 2018년 6월 러브 유어셀프 전 티어로 ‘빌보드 200’ 1위에 처음 올랐다. 이후 2018년 9월 러브 유어셀프 결 앤서 지난해 4월 맵 오브 더 솔 페르소나에 이어 이번 앨범

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


...

sim_count:  7.0
article index:  344
2020-08-23 00:00:00
BTS 다이너마이트 신기록 폭발최단 시간 1억뷰 달성스포티파이 1위 

 ‘다이너마이트’ 뮤직비디오는 공개 24시간30분 만에 조회수 1억뷰을 돌파했다. 유튜브 뮤직비디오 사상 방탄소년단 뮤직비디오 사상 가장 빠른 기록이다.  지난 22일 소속사 빅히트엔터테인먼트는 방탄소년단이 전날 오후 1시 공개한 ‘다이너마이트’ 뮤직비디오가 이날 오후 1시30분쯤 유튜브에서 조회수 1억뷰를 넘겼다고 밝혔다. 공개된 지 약 24시간30분 만에 세운 기록이다. 이는 지난 6월 32시간 만에 1억뷰를 달성하며 유튜브 뮤직비디오 사상 최단 기록을 세웠던 블랙핑크 ‘하우 유 라이크 댓How You Like That’보다 약 7시간 앞당긴 기록이다. 앞서 방탄소년단 사상 최단 시간 1억뷰 달성 뮤직비디오로 기록된 ‘작은 것들을 위한 시Boy With Luv’의 37시간37분보다도 13시간 빠르다.  앞서 ‘다이너마이트’ 뮤직비디오는 공개 시점에 동시 접속자 수 300만명을 넘겨 유튜브 프리미어 뮤직비디오 역대 최고 시청 기록을 달성하기도 했다. 아울러 공개 20여 분 만에 1000만뷰 1시간14분 만에 2000만뷰 3시간 만에 3000만뷰를 넘기는 등 빠른 속도로 조회수를 높였다. ‘다이너마이트’ 뮤직비디오까지 1억뷰를 찍으면서 방탄소년단은 총 26편의 억대뷰 뮤직비디오를 보유하게 됐다. 이는 국내 가수 중 최다 기록이다.  방탄소년단은 세계 최대 음악 스트리밍 업체인 스포티파이에서도 ‘신기록’을 써나가고 있다. 지난 22일 스포티파이가 발표한 최신 차트21일자에 따르면 전날 공개된 방탄소년단의 ‘다이너마이트’는 ‘글로벌 톱 50’ 차트에서 1위로 진입했다. 스포티파이 글로벌 톱 50 정상에 오른 한국 가수는 방탄소년단이 처음이다. ‘다이너마이트’는 발매 첫날 전 세계적으로 777만8950회 스트리밍돼 올해 스포티파이 집계에서 발매 첫날 최다 글로벌 스트리밍 기록을 세웠다. 앞서 지난

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


sim_count:  23.0
article index:  626
2020-10-13 00:00:00
NYT 중국 네티즌 BTS 악의 없는 발언에도 공격 

 미국 뉴욕타임스NYT는 12일현지 시각 “방탄소년단BTS의 악의 없는 발언에 중국 네티즌들이 공격적으로 반응했다”고 보도했다. BTS의 리더 RM본명 김남준은 최근 수상 소감에서 “6·25 전쟁 때 한국과 미국이 함께 시련을 겪었다”고 밝혔는데 이를 두고 중국 네티즌들은 “중국군 희생을 무시했다”며 비난을 쏟아냈다.  NYT는 이날 ‘BTS는 한국전쟁 희생자를 기렸으나 일부 중국인은 모욕으로 여겼다’라는 제목의 기사에서 “도발적인 언사보다는 포용성으로 잘 알려진 인기 보이밴드 BTS의 리더가 최근 한국전쟁 기념식에서 미국과 한국이 겪은 시련을 인정한다는 악의 없는 발언을 했다”면서 “그러나 중국 네티즌들은 곧바로 BTS를 공격하는 글을 온라인에 올렸다”고 썼다.  앞서 BTS는 지난 7일현지 시각 미국의 한·미 친선 비영리재단인 코리아소사이어티의 연례행사에서 한·미 우호 관계 증진에 공을 세운 한국인과 미국인에게 주는 밴플리트상을 받았다. 이 상은 미8군 사령관으로 6·25 전쟁에 참전했고 1957년 코리아소사이어티를 창립한 제임스 밴 플리트 장군을 기리기 위해 1995년 제정됐다. 리더 RM은 온라인으로 진행된 시상식에서 “올해는 한국전쟁 70주년으로 우리는 양국한·미이 함께 겪었던 고난의 역사와 많은 남성과 여성의 희생을 영원히 기억해야 한다”는 말이 포함된 소감을 밝혔다. 해당 내용은 소감문 전체의 10 미만이었다.  그러나 일부 중국 네티즌은 ‘양국이 겪었던 고난의 역사’라는 표현만 짚어 문제를 삼으며 “중국인이 큰 희생을 하며 미군을 막아줬는데 어떻게 이를 무시할 수 있느냐”고 비난하기 시작했다. 중국 광저우에 사는 졸리 류21는 NYT 인터뷰에서 “우리는 BTS에게 우리와 같은 정치적 견해를 가지도록 강요할 수는 없지만 BTS가 중국에서 돈을 벌고 있다는 점에서 BTS는 모든 국가를 존중해야 

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


sim_count:  10.0
article index:  803
2020-12-01 00:00:00
한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다 

 빌보드는 지난달 30일현지시간 방탄소년단의 신곡 ‘라이프 고스 온Life Goes O’이 빌보드 싱글 차트인 ‘핫 100’ 1위로 핫샷 데뷔진입과 동시에 1위했다고 밝혔다.  앞서 방탄소년단은 지난 8월 발표한 영어곡 ‘다이너마이트’ 제이슨 데룰로·조시 685와 함께 부른 ‘새비지 러브’로 ‘핫 100’ 차트 1위를 차지했다. ‘새비지 러브’에도 일부 한국어 가사가 등장하지만 가사 대부분은 영어였다. 가사가 한국어인 곡으로 차트 정상을 차지한 것은 이번이 처음이다. 빌보드는 “비영어 곡이 ‘핫 100’ 1위를 한 것은 2017년 루이스 폰시와 대디 양키가 발매한 스페인어 곡 ‘데스파시토’ 이후 처음”이라고 밝혔다.  ‘라이프 고스 온’의 라디오 방송 횟수는 상대적으로 저조한 편이었다. 빌보드가 인용한 닐슨뮤직 데이터에 따르면 이 곡은 발매 첫주2329일 41만명의 라디오 청취자에게 노출됐다. 8730만명에게 노출된 ‘핫 100’ 2위 24k골든과 이안 디올의 ‘무드’는 물론 1160만명에게 노출된 방탄소년단의 전작 ‘다이너마이트’와도 차이가 컸다.  하지만 스트리밍 1490만회 판매량 15만건이라는 압도적 음원 판매량이 이를 벌충했다. 이규탁 한국조지메이슨대 교수는 “핫 100 차트 1위를 위해선 라디오 방송 횟수가 중요한데 라디오는 비영어 곡에 대한 장벽이 워낙 높은 편이다. 음원 판매로 이를 메웠다는 것은 방탄소년단의 팬덤이 그만큼 커지고 단단해졌다는 뜻”이라고 말했다. 김윤하 음악평론가 역시 “‘다이너마이트’의 성공이 더 많은 팬을 유입시키고 높은 인지도를 형성했고 그 결과가 두 번째 빌보드 차트 1위로 이어졌다. 대중과 방탄소년단 사이를 좁히는 또 하나의 요인이 만들어진 것”이라고 했다.  BE는 방탄소년단이 코로나19 시대를 살아가며 느낀 복잡한 감정을 음악으로 풀어낸 앨범